In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.Logs (
  id BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1) NOT NULL,
  tabela STRING,
  camada STRING,
  status STRING,
  origem STRING,
  destino STRING,
  temp_inicial TIMESTAMP,
  temp_final TIMESTAMP,
  linhas_origem INT,
  linhas_processadas INT
);


In [0]:
import datetime

def addlog(log_tabela, log_camada, log_origem, log_destino, log_status, atualizacao):
    '''
    Log de processamento
    - Mapeia o processamento da arquiteura medalhao
    - Adiciona o log de processamento
    - Atualiza o log de processamento
    '''
    if log_status is None:
        log_status = 'null'

    log_tempo = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_linhas_origem = spark.read.table(f"{log_origem}.{log_tabela}").count()
    try:
        log_linhas_destino = spark.read.table(f"{log_destino}.{log_tabela}").count()
    except:
        log_linhas_destino = 'null'

    if atualizacao == 0:
        query = f"""
            INSERT INTO default.logs 
            (tabela, camada, status, origem, destino, temp_inicial, temp_final, linhas_origem, linhas_processadas) 
            VALUES ('{log_tabela}', '{log_camada}', 'Em processamento','{log_origem}', '{log_destino}', '{log_tempo}', null, {log_linhas_origem}, {log_linhas_destino})
        """
        spark.sql(query)
    
    elif atualizacao == 1:
        
        query = f"""
            UPDATE default.logs 
            SET temp_final = '{log_tempo}',
                linhas_processadas = {log_linhas_destino},
                status = '{log_status}'
             WHERE id = (select id from default.logs where status = 'Em processamento' order by temp_inicial desc limit 1)
 
        """
        spark.sql(query)